Step 1: Setup

We need to specify:

How often the bot runs

Which market data it uses

In [ ]:
from datetime import datetime, timedelta

from investing_algorithm_framework import (
    CCXTOHLCVMarketDataSource,
    CCXTTickerMarketDataSource
)

bitvavo_btc_eur_ohlcv_2h = CCXTOHLCVMarketDataSource(
    identifier="BTC/EUR-ohlcv",
    market="BITVAVO",
    symbol="BTC/EUR",
    timeframe="2h",
    # retrieve data from the last ~17 days
    start_date_func=lambda: datetime.utcnow() - timedelta(days=17)
)

# Ticker data to track orders, trades and positions
bitvavo_btc_eur_ticker = CCXTTickerMarketDataSource(
    identifier="BTC/EUR-ticker",
    market="BITVAVO",
    symbol="BTC/EUR",
)


### Simulation sandbox -> backtrader (python package)